## Clean and plot

In [192]:
import pandas as pd
import  json
from bokeh.palettes import Spectral4
from bokeh.plotting import figure, show
from bokeh.embed import file_html
from bokeh.resources import CDN
from bokeh.models import CustomJS, Dropdown
import glob

Reading the data and cleaning:

In [193]:
csv_files = glob.glob('./data/*.csv')

for file in csv_files:
    df = pd.read_csv(file)

    df = df[df['time'] != 'Not enough data']
    
    try:
        df['date_recorded'] = df['date_recorded'].str.replace(',', '')
        df['time'] = df['time'].str.replace('months', '')
        df['time'] = df['time'].astype(int)
        df['date_recorded'] = pd.to_datetime(df['date_recorded'], format='%B %d %Y')

    except:
        pass

    # save the dataframe as a csv
    df.to_csv(file, index=False)


Reading the data / csv files and making each one a dataframe:

In [194]:
csv_files = glob.glob('./data/*.csv')

dfs = []

for file in csv_files:
    df = pd.read_csv(file, parse_dates=['date_recorded'])
    df["category"] = file.split('/')[-1].split('.')[0]
    df["category"] = df["category"].str.replace('_', ' ')
    print(df)
    dfs.append(df)


   time date_recorded                  category
0    22    2022-10-18  Skilled workers (Quebec)
1    23    2022-11-18  Skilled workers (Quebec)
2    22    2022-12-18  Skilled workers (Quebec)
3    21    2023-01-18  Skilled workers (Quebec)
4    22    2023-02-18  Skilled workers (Quebec)
5    21    2023-03-18  Skilled workers (Quebec)
6    20    2023-04-18  Skilled workers (Quebec)
   time date_recorded               category
0    67    2022-10-18  Quebec Business Class
1    65    2022-11-18  Quebec Business Class
2    68    2022-12-18  Quebec Business Class
3    64    2023-01-18  Quebec Business Class
4    64    2023-02-18  Quebec Business Class
5    65    2023-03-18  Quebec Business Class
6    67    2023-04-18  Quebec Business Class
   time date_recorded                   category
0    22    2022-10-18  Skilled workers (Federal)
1    24    2022-11-18  Skilled workers (Federal)
2    25    2022-12-18  Skilled workers (Federal)
3    24    2023-01-18  Skilled workers (Federal)
4    23    

In [195]:
# for each dataframe in dfs, change the name of the time column to the category name
for df in dfs:
    df.rename(columns={'time': df['category'][0]}, inplace=True)
    df.drop(['category'], axis=1, inplace=True)

In [202]:
dfs[0]["qc_skilled"] = dfs[0]["Skilled workers (Quebec)"]
dfs[0]["qc_business"] = dfs[1]["Quebec Business Class"]
dfs[0]["skilled_fed"] = dfs[2]["Skilled workers (Federal)"]
dfs[0]["atlantic"] = dfs[3]["Atlantic Immigration Program"]
dfs[0]["self_employed"] = dfs[4]["Self-employed persons (Federal)"]
dfs[0]["startup"] = dfs[5]["Start-up visa"]
dfs[0]["trades"] = dfs[6]["Skilled trades (Federal)"]
dfs[0]["can_exp"] = dfs[7]["Canadian Experience Class"]

In [208]:
big_frame = dfs[0]

In [210]:
big_frame

,Skilled workers (Quebec),date_recorded,qc_skilled,qc_business,skilled_fed,atlantic,self_employed,startup,trades,can_exp
0,22,2022-10-18,22,67,22,8,48,30,39.0,4
1,23,2022-11-18,23,65,24,6,60,41,38.0,5
2,22,2022-12-18,22,68,25,6,64,42,38.0,5
3,21,2023-01-18,21,64,24,4,85,44,39.0,8
4,22,2023-02-18,22,64,23,4,24,45,40.0,4
5,21,2023-03-18,21,65,22,4,58,34,48.0,5
6,20,2023-04-18,20,67,27,4,44,35,NaN,5


In [211]:
# drop skill workers (quebec) column

big_frame.drop(['Skilled workers (Quebec)'], axis=1, inplace=True)

In [221]:
from bokeh.models import ColumnDataSource, CustomJS, Range1d, Select
from bokeh.plotting import figure, output_notebook, show
from bokeh.layouts import column

import pandas as pd
import numpy as np

# fix the y_range to be the same for two lines
p = figure(y_range=Range1d(0, 60), x_axis_type='datetime', width=600, height=400)

# shared datasource
source = ColumnDataSource(big_frame)

plot_1 = p.line(x="date_recorded", y="qc_skilled", color="teal", source=source, line_width=2)
plot_2 = p.line(x="date_recorded",  y="can_exp", color="firebrick", source=source, line_width=2)

# initialise the plot with only y1 visible - to match the dropdown default
plot_2.visible = False

# dropdown widget + Javascript code for interactivity
select = Select(title="Program:", value="Line 1", options=["Quebec Skilled Workers", "Canadian Experience Class"])
callback = CustomJS(args=dict(line_1=plot_1, line_2=plot_2), code="""
line_1.visible = true
line_2.visible = true

if (this.value === "Line 1") {
    line_2.visible = false 
} else {
    line_1.visible = false
}
""")
select.js_on_change("value", callback)

# save the JS code to a file
with open('plot4.js', 'w') as f:
    f.write(callback.code)

layout = column(select, p)

# save the HTML file
html = file_html(layout, CDN, "my plot")
with open('plot4.html', 'w') as f:
    f.write(html)


In [ ]:
from bokeh.embed import file_html
from bokeh.resources import CDN

# generate the HTML file
html = file_html(layout, CDN, "my plot")

# save the HTML file to disk
with open("my_plot.html", "w") as f:
    f.write(html)